In [6]:
#Define the col names
colnames=["sepal_length_in_cm", "sepal_width_in_cm","petal_length_in_cm","petal_width_in_cm", "class"]

#Read the dataset
dataset = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header = None, names= colnames )

#Data
dataset.head()

#Encoding the categorical column
dataset = dataset.replace({"class":  {"Iris-setosa":1,"Iris-versicolor":2, "Iris-virginica":3}})
#Visualize the new dataset
dataset.head()


X = dataset.iloc[:,:-1]
y = dataset.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

#make splits
split1 = X_train.drop(columns=['sepal_length_in_cm','sepal_width_in_cm'])
split2 = X_train.drop(columns=['petal_length_in_cm','petal_width_in_cm'])

split1_test = X_test.drop(columns=['sepal_length_in_cm','sepal_width_in_cm'])
split2_test = X_test.drop(columns=['petal_length_in_cm','petal_width_in_cm'])


MODEL 1


In [9]:
#Create the SVM model
from sklearn.svm import SVC
classifier1 = SVC(kernel = 'linear', random_state = 0)
#Fit the model for the data

classifier1.fit(split1, y_train)

#Make the prediction
y_pred = classifier1.predict(split1_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier1, X = split1, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]
Accuracy: 97.27 %
Standard Deviation: 4.17 %


<BR>

MODEL 2

In [10]:
#Create the SVM model
from sklearn.svm import SVC
classifier2 = SVC(kernel = 'linear', random_state = 0)
#Fit the model for the data

classifier2.fit(split2, y_train)

#Make the prediction
y_pred = classifier2.predict(split2_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier2, X = split2, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

[[13  0  0]
 [ 0 11  5]
 [ 0  4  5]]
Accuracy: 81.29 %
Standard Deviation: 13.68 %


In [11]:
models=[classifier1,classifier2]
splits=[split1_test,split2_test]

COINMPRO FROM HERE

In [ ]:
#SyMPC imports required for encrypted inference
import sympc
from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor
from sympc.protocol import Falcon,FSS

In [ ]:
def get_clients(n_parties):
    parties=[]
    for index in range(n_parties): 
        parties.append(sy.VirtualMachine(name = "worker"+str(index)).get_root_client())
    return parties

In [ ]:
def split_send(data,session):
    """Splits data into number of chunks equal to number of parties and distributes it to respective 
       parties.
    """
    data_pointers = []
    
    split_size = int(len(data)/len(session.parties))+1
    for index in range(0,len(session.parties)):
        ptr=data[index*split_size:index*split_size+split_size].share(session=session)
        data_pointers.append(ptr)
        
    return data_pointers

In [ ]:
def inference(n_clients,Models,splits,protocol=None):
    # Get VM clients 
    parties=get_clients(n_clients)

  # Setup the session for the computation
    if(protocol):
        session = Session(parties = parties,protocol = protocol)
    else:
        session = Session(parties = parties)
        
    SessionManager.setup_mpc(session)
    pointers=[]
    mpc_model=[]

    
    for i in range(len(Models)):
        #Split data and send data to clients
        pointers.append(split_send(splits[i],session))

        #Encrypt model 
        mpc_model.append(m[i].share(session))
        
        
        
   #INFERENCE FROM HERE
    start_time = time.time()
    
    for i in range(len(Models)): 
        results = []
        for j in range(len(pointers[0])):
            encrypted_results = mpc_model[i](pointers[i][j])  #may b with .predict
            plaintext_results = encrypted_results.reconstruct()
            results.append(plaintext_results)
        predictions = torch.cat(results).reshape([-1])
        
    end_time = time.time()

    print(f"Time for inference: {end_time-start_time}s")
    
    

    #Calculate Loss
    for i in range(len(Models)):
        accuracies = cross_val_score(estimator = Models[i], X = splits[i], y = y_train, cv = 10)
        print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
        print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))
#     return predictions